In [ ]:
%load_ext autoreload

%autoreload 2

In [ ]:
from environment import *

import setting

path_dict = path(setting)

In [ ]:
gps_map = ccal.load_gps_map(path_dict['gps_map_file_path'])

In [ ]:
feature_dicts = make_feature_dicts(load_ccle=True)

In [ ]:
with open('../data/highlight.json') as json_file:
    
    highlight = json.load(json_file)

In [ ]:
for feature_group, features in highlight.items():
    
    if feature_group in feature_dicts:
    
        df = feature_dicts[feature_group]['df']

        annotation_type = feature_dicts[feature_group]['data_type']

        for feature in features:
            
            if feature in df.index:

                gps_map.plot_gps_map(
                    'h',
                    annotation_x_element=df.loc[feature].to_frame().T,
                    annotation_std_maxs=(3, ),
                    annotation_types=(annotation_type, ),
                    title='({}) {}'.format(
                        feature_group,
                        feature,
                    ),
                    element_marker_size=setting.GPS_MAP_H_ELEMENT_MARKER_SIZE,
                    html_file_path='{}/{} {}'.format(
                        path_dict['h_map_directory_path'],
                        feature_group,
                        feature,
                    ),
#                     plotly_file_path='{}/{} {}'.format(
#                         path_dict['h_state_maps_plotly_directory_path'],
#                         feature_group,
#                         feature,
#                     ),
                )